# 💀 Life Tables & Survivorship Curves
## Understanding Mortality Patterns Across the Lifespan

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-2-population/notebooks/02_life_tables_survivorship.ipynb)

---

> *"In this world nothing can be said to be certain, except death and taxes."* - Benjamin Franklin

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Construct and interpret **life tables**
2. Calculate **survivorship (lₓ)** and **mortality (qₓ)**
3. Understand **Type I, II, and III survivorship curves**
4. Calculate **life expectancy (eₓ)**
5. Distinguish **cohort vs static life tables**
6. Analyze **age-specific fecundity**
7. Apply life table analysis to real populations

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("✅ Ready to explore life tables!")
print("💀 Let's analyze mortality patterns!")

---

## 📚 Part 1: What is a Life Table?

### Definition:

A **life table** is a summary of age-specific mortality and survivorship in a population.

### Key Components:

| Symbol | Name | Definition |
|--------|------|------------|
| **x** | Age class | Age interval (e.g., 0-1, 1-2 years) |
| **nₓ** | Number alive | Individuals alive at start of age x |
| **dₓ** | Deaths | Individuals dying during age x |
| **qₓ** | Mortality rate | Probability of dying: qₓ = dₓ/nₓ |
| **lₓ** | Survivorship | Proportion surviving to age x: lₓ = nₓ/n₀ |
| **eₓ** | Life expectancy | Expected future lifespan from age x |
| **mₓ** | Fecundity | Average offspring per individual at age x |

### Two Types of Life Tables:

#### **1. Cohort Life Table** (Dynamic)
- Follow a **single cohort** from birth to death
- Track individuals over time
- **Advantage**: True mortality rates
- **Disadvantage**: Takes entire lifespan to complete
- **Example**: Follow 1000 babies born in 2000 until all die

#### **2. Static Life Table** (Cross-sectional)
- Sample population at **one point in time**
- Assumes stable age distribution
- **Advantage**: Quick to construct
- **Disadvantage**: Assumes constant conditions
- **Example**: Count skeletons of different ages

### Basic Formulas:

**Survivorship**:
```
lₓ = nₓ / n₀
```

**Mortality rate**:
```
qₓ = dₓ / nₓ = (nₓ - nₓ₊₁) / nₓ
```

**Life expectancy** (simplified):
```
eₓ = Σ(lᵢ) / lₓ    for i from x to max age
```

---

## 🦌 Part 2: Example - Dall Sheep Life Table

### Classic Study: Dall Mountain Sheep (Alaska)

**Method**: Static life table from skulls
- Collected 608 sheep skulls
- Aged by horn rings (like tree rings)
- Constructed life table

**Famous because**: One of the first published life tables for wild animals (Murie, 1944)

In [ ]:
# Dall sheep life table data
dall_sheep = pd.DataFrame({
    'Age_x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    'nx': [1000, 944, 887, 849, 803, 760, 699, 640, 571, 439, 252, 96, 6, 3, 0]
})

# Calculate life table components
n0 = dall_sheep['nx'].iloc[0]

# Deaths in each age class
dall_sheep['dx'] = dall_sheep['nx'].diff(-1).fillna(dall_sheep['nx'].iloc[-1])

# Survivorship (proportion surviving)
dall_sheep['lx'] = dall_sheep['nx'] / n0

# Mortality rate (probability of dying)
dall_sheep['qx'] = dall_sheep['dx'] / dall_sheep['nx']
dall_sheep['qx'] = dall_sheep['qx'].fillna(0)  # Handle last row

# Life expectancy (simplified calculation)
def calculate_life_expectancy(lx_series):
    ex = []
    for i in range(len(lx_series)):
        future_lx = lx_series[i:].sum()
        ex.append(future_lx / lx_series[i] if lx_series[i] > 0 else 0)
    return ex

dall_sheep['ex'] = calculate_life_expectancy(dall_sheep['lx'])

# Display life table
print("\n🦌 Dall Sheep Life Table (Murie, 1944):\n")
print(dall_sheep.round(3).to_string(index=False))

# Key statistics
print("\n\n📊 Key Statistics:")
print(f"\n   Starting cohort (n₀): {n0}")
print(f"   Life expectancy at birth (e₀): {dall_sheep['ex'].iloc[0]:.2f} years")
print(f"   Maximum age observed: {dall_sheep['Age_x'].max()} years")
print(f"\n   Age with highest mortality: {dall_sheep.loc[dall_sheep['qx'].idxmax(), 'Age_x']} years")
print(f"   Highest mortality rate: {dall_sheep['qx'].max():.3f} ({dall_sheep['qx'].max()*100:.1f}%)")
print(f"\n   Survival to age 1: {dall_sheep['lx'].iloc[1]:.3f} ({dall_sheep['lx'].iloc[1]*100:.1f}%)")
print(f"   Survival to age 5: {dall_sheep['lx'].iloc[5]:.3f} ({dall_sheep['lx'].iloc[5]*100:.1f}%)")
print(f"   Survival to age 10: {dall_sheep['lx'].iloc[10]:.3f} ({dall_sheep['lx'].iloc[10]*100:.1f}%)")

In [ ]:
# Visualize Dall sheep survivorship and mortality
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Number Alive (nₓ)',
        'Survivorship Curve (lₓ)',
        'Deaths per Age (dₓ)',
        'Mortality Rate (qₓ)'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.12
)

# Number alive
fig.add_trace(
    go.Scatter(
        x=dall_sheep['Age_x'],
        y=dall_sheep['nx'],
        mode='lines+markers',
        line=dict(width=3, color='blue'),
        marker=dict(size=8),
        fill='tozeroy',
        fillcolor='rgba(0,0,255,0.2)'
    ),
    row=1, col=1
)

# Survivorship (log scale)
fig.add_trace(
    go.Scatter(
        x=dall_sheep['Age_x'],
        y=dall_sheep['lx'],
        mode='lines+markers',
        line=dict(width=3, color='green'),
        marker=dict(size=8)
    ),
    row=1, col=2
)

# Deaths
fig.add_trace(
    go.Bar(
        x=dall_sheep['Age_x'],
        y=dall_sheep['dx'],
        marker_color='red',
        opacity=0.7
    ),
    row=2, col=1
)

# Mortality rate
fig.add_trace(
    go.Scatter(
        x=dall_sheep['Age_x'],
        y=dall_sheep['qx'],
        mode='lines+markers',
        line=dict(width=3, color='darkred'),
        marker=dict(size=8),
        fill='tozeroy',
        fillcolor='rgba(139,0,0,0.2)'
    ),
    row=2, col=2
)

# Update axes
fig.update_xaxes(title_text="Age (years)", row=1, col=1)
fig.update_xaxes(title_text="Age (years)", row=1, col=2)
fig.update_xaxes(title_text="Age (years)", row=2, col=1)
fig.update_xaxes(title_text="Age (years)", row=2, col=2)

fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_yaxes(title_text="Proportion", type='log', row=1, col=2)  # Log scale!
fig.update_yaxes(title_text="Deaths", row=2, col=1)
fig.update_yaxes(title_text="Probability", row=2, col=2)

fig.update_layout(
    title="🦌 Dall Sheep Life Table Analysis<br><sub>Data from Murie (1944) - Alaska population</sub>",
    height=800,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n📈 Patterns Observed:\n")
print("   1. EARLY MORTALITY (Ages 0-1):")
print("      • ~6% die in first year (vulnerable lambs)")
print("      • High predation risk")
print("\n   2. LOW MORTALITY (Ages 1-8):")
print("      • Stable, low death rates (~5-10%/year)")
print("      • Prime adult years")
print("\n   3. SENESCENCE (Ages 9+):")
print("      • Mortality increases dramatically")
print("      • Age 10-11: 62% mortality!")
print("      • Old age effects (teeth wear, disease)")
print("\n💡 This is a TYPE I survivorship curve!")

---

## 📊 Part 3: The Three Types of Survivorship Curves

### Type I: Low Juvenile Mortality (Convex)

**Pattern**: Most survive to old age, then die

**Characteristics**:
- High parental care
- Few offspring
- Death mainly from senescence

**Examples**:
- Humans (developed countries)
- Elephants
- Dall sheep
- Large mammals with parental care

**Shape**:
```
1.0 |___________
    |            \
    |             \
    |              |
0.0 |______________|
     Young    Old
```

### Type II: Constant Mortality (Linear)

**Pattern**: Equal probability of death at any age

**Characteristics**:
- Random mortality
- No age preference
- Constant hazard

**Examples**:
- Birds (many species)
- Small mammals
- Lizards
- Adult hydra

**Shape**:
```
1.0 |\
    | \
    |  \
    |   \
0.0 |____\
     Young    Old
```

### Type III: High Juvenile Mortality (Concave)

**Pattern**: Massive early death, few reach adulthood

**Characteristics**:
- Little/no parental care
- MANY offspring produced
- "Lottery" survival

**Examples**:
- Fish (most species)
- Marine invertebrates
- Plants (trees)
- Insects
- Amphibians

**Shape**:
```
1.0 |
    ||
    ||________
    |
0.0 |_________
     Young    Old
```

### Visual Comparison:

On a **log scale** (standard for survivorship curves):

```
Log(lx)
  ↑
  |  Type I  (humans)
  |___________
  |    Type II (birds)  \
  |       \             |
  |        \            |
  |  Type III (fish)    |
  ||                    |
  ||___________________|
  |_____________________→ Age
```

In [ ]:
# Create idealized three types of survivorship curves
ages = np.linspace(0, 100, 101)

# Type I: Convex (humans, elephants)
type_I = np.where(ages < 70, 1.0, 1.0 - ((ages - 70) / 30)**2)
type_I = np.clip(type_I, 0, 1)

# Type II: Linear (birds)
type_II = 1.0 - (ages / 100)

# Type III: Concave (fish, trees)
type_III = np.exp(-0.08 * ages)

# Create comparison plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ages, y=type_I,
    mode='lines',
    line=dict(width=4, color='blue'),
    name='Type I (Humans, Elephants)<br>High survival until old age'
))

fig.add_trace(go.Scatter(
    x=ages, y=type_II,
    mode='lines',
    line=dict(width=4, color='green'),
    name='Type II (Birds, Lizards)<br>Constant mortality'
))

fig.add_trace(go.Scatter(
    x=ages, y=type_III,
    mode='lines',
    line=dict(width=4, color='red'),
    name='Type III (Fish, Trees)<br>High early mortality'
))

fig.update_layout(
    title="📊 Three Types of Survivorship Curves<br><sub>Plotted on log scale (standard presentation)</sub>",
    xaxis_title="Age (% of maximum lifespan)",
    yaxis_title="Survivorship (lₓ)",
    yaxis_type='log',
    height=600,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

print("\n📊 Survivorship Curve Types:\n")
print("   TYPE I (Blue - Convex):")
print("      • Pattern: ___________\\__")
print("      • Most survive to old age")
print("      • Death from senescence")
print("      • Examples: Humans, elephants, whales")
print("      • Strategy: K-selected")
print("\n   TYPE II (Green - Linear):")
print("      • Pattern: \\\\\\\\\\\\\\\\\\")
print("      • Constant mortality rate")
print("      • Age doesn't matter")
print("      • Examples: Birds, rodents, lizards")
print("      • Strategy: Intermediate")
print("\n   TYPE III (Red - Concave):")
print("      • Pattern: ||___________")
print("      • Massive juvenile mortality")
print("      • Few reach adulthood")
print("      • Examples: Fish, oysters, trees")
print("      • Strategy: r-selected")
print("\n💡 Most species are not pure types - they're intermediate!")

---

## 🐟 Part 4: Type III Example - Salmon Life Table

### Pacific Salmon Lifecycle:

**Extreme Type III survivorship!**

1. **Eggs laid**: 3,000-7,000 per female
2. **Eggs survive**: ~30-50% (predation, disease)
3. **Hatch to fry**: ~1,500 alive
4. **Fry to smolt**: ~150 survive (10%)
5. **Smolt to ocean**: ~75 survive (5%)
6. **Return to spawn**: ~5 survive (0.17%!)

**Result**: 99.83% mortality!

In [ ]:
# Salmon life table
salmon = pd.DataFrame({
    'Life_Stage': ['Eggs', 'Alevin', 'Fry', 'Parr', 'Smolt', 'Ocean (1yr)', 'Ocean (2yr)', 'Adults Return'],
    'Age_months': [0, 2, 4, 12, 18, 30, 42, 48],
    'nx': [3000, 1500, 600, 300, 150, 75, 20, 5],
    'Location': ['Stream', 'Stream', 'Stream', 'Stream', 'Estuary', 'Ocean', 'Ocean', 'Stream']
})

# Calculate life table
n0 = salmon['nx'].iloc[0]
salmon['lx'] = salmon['nx'] / n0
salmon['dx'] = salmon['nx'].diff(-1).fillna(salmon['nx'].iloc[-1])
salmon['qx'] = (salmon['dx'] / salmon['nx']).fillna(0)
salmon['Percent_Survive'] = (salmon['lx'] * 100).round(2)

# Display table
print("\n🐟 Pacific Salmon Life Table (Typical):\n")
display_cols = ['Life_Stage', 'Age_months', 'nx', 'lx', 'Percent_Survive', 'Location']
print(salmon[display_cols].to_string(index=False))

print("\n\n💀 Mortality Summary:")
print(f"\n   Starting eggs: {n0:,}")
print(f"   Returning adults: {salmon['nx'].iloc[-1]}")
print(f"   Overall survival: {salmon['lx'].iloc[-1]*100:.3f}%")
print(f"   Overall mortality: {(1-salmon['lx'].iloc[-1])*100:.2f}%")
print("\n   Stage-specific mortality:")
for i, row in salmon.iterrows():
    if i < len(salmon) - 1:
        print(f"      {row['Life_Stage']:15} → {salmon.iloc[i+1]['Life_Stage']:15}: {row['qx']*100:5.1f}% die")

In [ ]:
# Visualize salmon survivorship (Type III)
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Survivorship Curve (log scale)', 'Number Alive (linear)'),
    horizontal_spacing=0.12
)

# Survivorship curve (log scale)
fig.add_trace(
    go.Scatter(
        x=salmon['Age_months'],
        y=salmon['lx'],
        mode='lines+markers',
        line=dict(width=4, color='red'),
        marker=dict(size=12, symbol='diamond'),
        text=salmon['Life_Stage'],
        hovertemplate='<b>%{text}</b><br>Age: %{x} months<br>Survival: %{y:.4f}<extra></extra>'
    ),
    row=1, col=1
)

# Number alive (linear)
fig.add_trace(
    go.Bar(
        x=salmon['Life_Stage'],
        y=salmon['nx'],
        marker_color='orange',
        text=salmon['nx'],
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Number: %{y}<extra></extra>'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Age (months)", row=1, col=1)
fig.update_xaxes(title_text="Life Stage", tickangle=45, row=1, col=2)
fig.update_yaxes(title_text="Survivorship (lₓ)", type='log', row=1, col=1)
fig.update_yaxes(title_text="Number Alive", row=1, col=2)

fig.update_layout(
    title="🐟 Salmon Type III Survivorship<br><sub>Extreme juvenile mortality - only 0.17% survive!</sub>",
    height=500,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n🐟 Why Such High Mortality?\n")
print("   EGGS (50% die):")
print("      • Fungal infections")
print("      • Low oxygen in gravel")
print("      • Predation by other fish")
print("\n   FRY (60% die):")
print("      • Birds (herons, kingfishers)")
print("      • Larger fish")
print("      • Poor swimming ability")
print("\n   SMOLT (50% die):")
print("      • Transition to saltwater stressful")
print("      • Predation increases")
print("      • Estuarine predators")
print("\n   OCEAN (73% die):")
print("      • Seals, sea lions")
print("      • Orcas")
print("      • Sharks")
print("      • Commercial fishing")
print("\n💡 Evolutionary Strategy:")
print("   Produce THOUSANDS of eggs because so few survive!")
print("   Classic r-selection: Quantity over quality")

---

## 👥 Part 5: Human Survivorship - Then vs Now

### Historical Change:

Human survivorship has dramatically improved!

**1900 (Developing)**:
- High infant mortality
- Type II-III (high early death)
- Life expectancy: ~30-40 years

**2020 (Developed)**:
- Low infant mortality
- Type I (most reach old age)
- Life expectancy: ~80 years

In [ ]:
# Compare human survivorship: historical vs modern
ages_human = np.array([0, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

# 1900 (developing country - higher mortality)
lx_1900 = np.array([1.0, 0.75, 0.65, 0.62, 0.60, 0.55, 0.48, 0.40, 0.28, 0.15, 0.05, 0.01, 0.001])

# 2020 (developed country - low mortality)
lx_2020 = np.array([1.0, 0.995, 0.993, 0.992, 0.990, 0.988, 0.985, 0.975, 0.950, 0.850, 0.550, 0.150, 0.010])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ages_human, y=lx_1900,
    mode='lines+markers',
    line=dict(width=3, color='red', dash='dash'),
    marker=dict(size=10),
    name='1900 (Developing Country)'
))

fig.add_trace(go.Scatter(
    x=ages_human, y=lx_2020,
    mode='lines+markers',
    line=dict(width=4, color='blue'),
    marker=dict(size=10),
    name='2020 (Developed Country)'
))

fig.update_layout(
    title="👥 Human Survivorship: 1900 vs 2020<br><sub>Dramatic improvement in survival rates</sub>",
    xaxis_title="Age (years)",
    yaxis_title="Survivorship (lₓ)",
    yaxis_type='log',
    height=600,
    template='plotly_white'
)

fig.show()

# Calculate statistics
print("\n👥 Human Survivorship Comparison:\n")
print("   1900 (Historical):")
print(f"      • Infant survival (age 1): {lx_1900[1]*100:.1f}%")
print(f"      • Survive to age 50: {lx_1900[7]*100:.1f}%")
print(f"      • Survive to age 70: {lx_1900[9]*100:.1f}%")
print(f"      • Life expectancy: ~40 years")
print("\n   2020 (Modern):")
print(f"      • Infant survival (age 1): {lx_2020[1]*100:.1f}%")
print(f"      • Survive to age 50: {lx_2020[7]*100:.1f}%")
print(f"      • Survive to age 70: {lx_2020[9]*100:.1f}%")
print(f"      • Life expectancy: ~80 years")
print("\n💡 What Changed?")
print("   • Vaccines (reduced child mortality)")
print("   • Antibiotics (infectious disease control)")
print("   • Sanitation (clean water, sewage)")
print("   • Nutrition (food security)")
print("   • Medical care (surgery, diagnostics)")
print("\n📊 Curve Type Shift:")
print("   1900: Type II-III (high early mortality)")
print("   2020: Type I (survival to old age)")

---

## 🌸 Part 6: Fecundity & Reproductive Value

### Adding Reproduction to Life Tables:

So far we've only looked at mortality. But **reproduction** is equally important!

### Additional Columns:

| Symbol | Name | Definition |
|--------|------|------------|
| **mₓ** | Fecundity | Average offspring per individual at age x |
| **lₓmₓ** | Reproductive output | Survivors × fecundity |
| **R₀** | Net reproductive rate | Σ(lₓmₓ) = lifetime offspring |
| **G** | Generation time | Average age of reproduction |

### Net Reproductive Rate (R₀):

```
R₀ = Σ(lₓ × mₓ)
```

**Interpretation**:
- **R₀ > 1**: Population growing
- **R₀ = 1**: Population stable (replacement)
- **R₀ < 1**: Population declining

### Generation Time:

```
G = Σ(x × lₓ × mₓ) / R₀
```

Average age at which offspring are produced

In [ ]:
# Example: Life table with fecundity (hypothetical mammal)
mammal_life_table = pd.DataFrame({
    'Age_x': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'lx': [1.00, 0.80, 0.70, 0.60, 0.45, 0.25, 0.10, 0.03, 0.00],
    'mx': [0.0, 0.0, 1.5, 3.0, 3.5, 2.5, 1.0, 0.0, 0.0]  # Offspring per female
})

# Calculate reproductive output
mammal_life_table['lx_mx'] = mammal_life_table['lx'] * mammal_life_table['mx']
mammal_life_table['x_lx_mx'] = mammal_life_table['Age_x'] * mammal_life_table['lx_mx']

# Net reproductive rate
R0 = mammal_life_table['lx_mx'].sum()

# Generation time
G = mammal_life_table['x_lx_mx'].sum() / R0

# Display table
print("\n🦊 Life Table with Fecundity (Hypothetical Mammal):\n")
print(mammal_life_table.round(3).to_string(index=False))

print("\n\n📊 Reproductive Statistics:")
print(f"\n   Net Reproductive Rate (R₀): {R0:.3f}")
if R0 > 1:
    print(f"      → Population is GROWING")
elif R0 < 1:
    print(f"      → Population is DECLINING")
else:
    print(f"      → Population is STABLE")
print(f"\n   Generation Time (G): {G:.2f} years")
print(f"      → Average age of reproduction")
print(f"\n   Per capita growth rate (r): {np.log(R0)/G:.4f} per year")
print(f"      → Approximate exponential growth rate")

# Visualize
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Survivorship (lₓ)', 'Fecundity (mₓ)', 'Reproductive Output (lₓmₓ)'),
    horizontal_spacing=0.1
)

# Survivorship
fig.add_trace(
    go.Scatter(x=mammal_life_table['Age_x'], y=mammal_life_table['lx'],
               mode='lines+markers', line=dict(color='blue', width=3),
               marker=dict(size=10)),
    row=1, col=1
)

# Fecundity
fig.add_trace(
    go.Bar(x=mammal_life_table['Age_x'], y=mammal_life_table['mx'],
           marker_color='green'),
    row=1, col=2
)

# Reproductive output
fig.add_trace(
    go.Bar(x=mammal_life_table['Age_x'], y=mammal_life_table['lx_mx'],
           marker_color='purple'),
    row=1, col=3
)

# Mark generation time
fig.add_vline(x=G, line_dash="dash", line_color="red",
             annotation_text=f"G = {G:.1f} years",
             row=1, col=3)

fig.update_xaxes(title_text="Age (years)")
fig.update_yaxes(title_text="Survivorship", row=1, col=1)
fig.update_yaxes(title_text="Offspring/female", row=1, col=2)
fig.update_yaxes(title_text="lₓmₓ", row=1, col=3)

fig.update_layout(
    title=f"🌸 Fecundity Schedule (R₀ = {R0:.2f})",
    height=400,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n💡 Key Insights:")
print("   • Peak reproduction at ages 3-5")
print("   • No reproduction before age 2 (juveniles)")
print("   • Reproductive output peaks at age 4")
print("   • Why? Most survivors × highest fecundity")
print("\n📈 For population growth:")
print("   • Protect reproductive adults (ages 2-6)")
print("   • These contribute most to R₀")

---

## 🎓 Summary

### Key Takeaways:

✅ **Life tables**: Summarize age-specific mortality and survivorship  
✅ **Survivorship (lₓ)**: Proportion surviving to age x  
✅ **Mortality rate (qₓ)**: Probability of dying during age x  
✅ **Life expectancy (eₓ)**: Expected future lifespan from age x  
✅ **Type I, II, III**: Three patterns of mortality  
✅ **Fecundity (mₓ)**: Age-specific reproduction  
✅ **R₀**: Net reproductive rate (lifetime offspring)  

### The Three Types:

| Type | Shape | Mortality | Examples |
|------|-------|-----------|----------|
| **I** | Convex | Low juvenile, high senescence | Humans, elephants |
| **II** | Linear | Constant at all ages | Birds, lizards |
| **III** | Concave | High juvenile, low adult | Fish, trees, oysters |

### Life Table Types:

**Cohort (Dynamic)**:
- Follow one cohort through time
- True age-specific rates
- Time-consuming

**Static (Cross-sectional)**:
- Sample all ages at once
- Assumes stable conditions
- Quick to construct

### Applications:

#### 🦌 **Wildlife Management**:
- Identify vulnerable age classes
- Set hunting quotas
- Monitor population health

#### 🐟 **Fisheries**:
- Determine optimal harvest size
- Protect reproductive classes
- Predict stock recovery

#### 👥 **Human Demographics**:
- Life insurance calculations
- Pension planning
- Healthcare resource allocation

#### 🌳 **Conservation**:
- Endangered species recovery
- Identify critical life stages
- Guide intervention strategies

### Important Formulas:

**Survivorship**: lₓ = nₓ / n₀  
**Mortality rate**: qₓ = dₓ / nₓ  
**Net reproductive rate**: R₀ = Σ(lₓ × mₓ)  
**Generation time**: G = Σ(x × lₓ × mₓ) / R₀  
**Growth rate**: r ≈ ln(R₀) / G  

### Next Steps:

In the next notebook, we'll explore:
- Age structure and population pyramids
- How age distribution affects growth
- Demographic transition
- Predicting future populations

---

<div align="center">

**Made with 💚 by The Pattern Hunter Team**

[📓 Previous: Population Growth](01_population_growth_models.ipynb) | 
[🏠 Unit 2 Home](../../) | 
[📓 Next: Age Structure](03_age_structure_demography.ipynb)

</div>